In [1]:
from rdkit import Chem
from serenityff.charge.tree.dash_tree import DASHTree

In [2]:
tree = DASHTree(num_processes=1)
# n=1 19.5
# n=2 30.6
# n=3 31.6

Loading DASH tree data


In [3]:
tree.tree_storage[32]

[(0,
  32,
  -1,
  -1,
  0.7424436807632446,
  [0,
   1,
   32089,
   32583,
   52229,
   62300,
   84477,
   87969,
   93259,
   120516,
   156163,
   173008,
   196098,
   196333,
   196999,
   197004,
   197048,
   197277,
   197283,
   197312,
   197313,
   251701,
   288599,
   288761,
   298606,
   299917,
   299918,
   300480,
   300575,
   320805,
   321300,
   321305,
   321335]),
 (1,
  83,
  0,
  1,
  0.957500159740448,
  [2,
   180,
   555,
   922,
   979,
   1230,
   1281,
   1285,
   1298,
   1354,
   1498,
   1566,
   9119,
   20202,
   31472,
   31622,
   32040,
   32073,
   32081]),
 (2, 98, 0, 1, 0.9406411051750183, [3]),
 (3, 78, 2, 2, 0.06487224251031876, [4, 141, 173, 174, 175]),
 (4, 83, 2, 1, 0.03715686872601509, [5, 39, 51, 133, 139, 140]),
 (5, 32, 2, 1, 0.033940959721803665, [6, 24, 34]),
 (6, 33, 5, 1, 0.034606557339429855, [7]),
 (7, 64, 5, 1, 0.034025318920612335, [8, 19, 22, 23]),
 (8, 26, 6, 1, 0.033290036022663116, [9]),
 (9, 27, 8, 4, 0.0342686213552951

In [ ]:
# int(id_counter), int(atom_type), int(con_atom), int(con_type), 
#           float(oldNode.attention), []children
(2, 98, 0, 1, 0.9406411051750183, [3]),
(3, 78, 2, 2, 0.06487224251031876, [4, 141, 173, 174, 175]),
(4, 83, 2, 1, 0.03715686872601509, [5, 39, 51, 133, 139, 140]),
(5, 32, 2, 1, 0.033940959721803665, [6, 24, 34]),
(6, 33, 5, 1, 0.034606557339429855, [7]),
(7, 64, 5, 1, 0.034025318920612335, [8, 19, 22, 23]),
(8, 26, 6, 1, 0.033290036022663116, [9]),
(9, 27, 8, 4, 0.03426862135529518, [10]),
(10, 27, 8, 4, 0.033310238271951675, [11]),
(11, 27, 10, 4, 0.03356257453560829, [12]),
(12, 27, 9, 4, 0.03284271061420441, [13]),
(13, 33, 0, 1, 0.004910393618047237, [14, 15, 16]),
(14, 26, 13, 1, 0.004782457370311022, []),
(15, 32, 13, 1, 0.004723169840872288, []),
(16, 33, 13, 1, 0.004681633319705725, [17, 18]),
(17, 33, 14, 1, 0.004527490120381117, []),
(18, 26, 14, 1, 0.004242870956659317, []),
(19, 33, 6, 1, 0.004869766067713499, [20, 21]),
(20, 33, 0, 1, 0.0047450256533920765, []),
(21, 26, 0, 1, 0.004067601170390844, []),

In [5]:
tree.data_storage[32]

,level,atom,con,conType,result,stdDeviation,attention,count
0,1,32,-1,-1,0.025406,0.183350,0.742676,923818
1,2,83,0,1,0.161499,0.104126,0.957520,97859
2,3,98,0,1,-0.227051,0.054169,0.940430,480
3,4,78,2,2,-0.227051,0.054169,0.064880,480
4,5,83,2,1,-0.225342,0.057190,0.037170,353
...,...,...,...,...,...,...,...,...
321368,5,34,3,1,-0.608398,0.016068,0.000902,6
321369,6,34,3,1,-0.608398,0.016068,0.000888,6
321370,7,34,3,1,-0.593750,0.002249,0.000884,3
321371,7,23,3,1,-0.622559,0.010010,0.000744,3


In [ ]:
mol = Chem.MolFromSmiles("c1ccccc1")
mol = Chem.AddHs(mol)

In [ ]:
tree.get_molecules_partial_charges(mol)

In [ ]:
%load_ext line_profiler
%lprun -f tree.get_molecules_partial_charges tree.get_molecules_partial_charges(mol)

In [ ]:
%lprun -f tree.match_new_atom tree.match_new_atom(0, mol)